### <font size="6" color='blue'>Réseau de Neurones sur la base de données MNIST </font> 

Dans ce Notebook nous allons voir comment implémenter une application capable de reconnaître et de déterminer quel chiffre est écrit à la main. La base de données MNIST s'appuie sur 70000 chiffres écrits à la main par des employés du bureau de recensement américain. Ces chiffres ont ensuite été numérisés, centrés et normalisés pour permettre une utilisation informatique. De nombreuses techniques de Machine Learning en développement ont commencé sur cet échantillon de données qui en fait un peu le "Hello World" du Machine Learning. 

Nous allons donc voir dans ce Notebook comment implémenter un réseau de neurones totalement connectés pour identifier un chiffre donné en entrée. 

## <font size="5" color='blue'> I - Importer les bibliothèques  </font>

Commencez par importer les bibliothèques dont vous aurez besoin par la suite

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random as rand

Pour une machine avec plusieurs cartes graphiques GPU, la commande suivante permet de signifier au code que l'on souhaite utiliser uniquement le GPU en position 0 ce qui peut être utile quand vous avez plusieurs applications qui tournent en parallèle.

Les lignes suivantes apportent des informations quant à la configuration des GPU dans votre système.

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

## <font size="5" color='blue'> II - Charger et mettre en forme le dataset MNIST  </font>

Avec la commande suivante vous demandez à Tensorflow de chercher en mémoire la base de données MNIST. Il va d'abord la télécharger depuis l'Internet avant de la mettre en mémoire en vu d'une utilisation future. 

Vous séparez alors le dataset en deux parties : 
* "train" va être utilisé pour entrainer le réseau de neurones, 
* "test" pour vérifier ses performances une fois que l'entrainement est terminé. 

Notez qu'il est très important, comme précisé dans la littérature, de bien mettre le set "test" de côté pour vraiment déterminer les performances du modèle sur des données nouvelles afin d'éviter le phénomène de sur-apprentissage "over-fitting". Vous avez dans chaque set deux parties : 
* la partie "x" correspond aux images numérisées avec des matrices de valeurs qui représentent des pixels (entre 0 et 255) qui sont ensuite normalisés par la troisième ligne (entre 0 et 1), 
* la partie "y" qui correspond à un vecteur avec les labels des images, c'est-à-dire le chiffre représenté sur l'image correspondante dans la partie "x".

Vous pouvez décommenter la partie grisée par Ctrl+/ pour afficher les différentes informations que l'on vient d'aborder. 

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0 , x_test / 255.0  

# print(x_train[0])
# print(y_train[0])
# plt.imshow(x_train[0], cmap=plt.cm.binary)
# plt.show()

## <font size="5" color='blue'> III - Construire le réseau de neurones  </font>

Dans cette partie, on construit à proprement parler le réseau de neurones. 
* Vous commencez par la couche externe celle qui reçoit la matrice de pixels de taille 28x28 pixels. Le "batch size" est une variable d'optimisation du problème ("hyperparameter") qui donne le nombre de couple (matrice, label) utilisé à chaque itération par le programme d'entraînement. La théorie montre qu'il existe une "batch size" optimale pour chaque problème mais dans notre cas, un grand nombre accélère le processus d'apprentissage et la borne supérieure n'est limitée que par les performances de votre GPU. 

* Puis, on construit la couche cachée avec 128 neurones entièrement reliés à ceux de la couche d'entrée. On choisit la fonction d'activation REctfied Linear Unit dans le processus de calcul de l'activation de chaque neurone. 

* Enfin la couche de sortie est composée de 10 neurones qui correspondent aux 10 chiffres à détecter. La fonction d'activation "softmax" permet de normaliser les sorties comme des probabilités : en effet, si vous les sommez vous obtenez 1. 

Enfin, vous paramétrez le processus d'apprentissage avec les fonctions qui vont être utilisées par l'optimisateur. 

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28), batch_size=2048),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(10, activation = 'softmax')
])

model.compile(optimizer = 'adam',  
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

## <font size="5" color='blue'> IV - Lancer l'apprentissage  </font>

Cette partie réalise l'apprentissage de la base de données MNIST par le réseau de neurones précédemment créé. Le réseau de neurones va utiliser les relations entre les matrices de pixels de "x_train" et les labels de "y_train" pour générer un modèle de connaissances utilisables par la suite sur de nouveaux exemples. 

Le réseau de neurones effectue 10 phases d'apprentissage ("nb_epochs") en utilisant à chaque fois 2048 couples (matrice, label), vous vous rappelez de "batch size" ? De plus à chaque itération, il utilise 10% des données que vous mettez à sa disposition pour évaluer les pertes ("losses") du modèle et sa précision ("accuracy"). Attention, en aucun cas vous ne devez utiliser les données de test pour la validation car sinon, le modèle absorbe les informations contenues dans ces données et vous ne disposez plus de données vierges pour évaluer les performances réelles du modèle en fin d'entrainement sur de nouvelles données. 

In [ ]:
nb_epochs=20

history = model.fit(x_train, y_train, epochs=nb_epochs, validation_split=0.1)

## <font size="5" color='blue'> V - Evaluer le modèle  </font>

Dans cette partie, vous utilisez les données test que vous avez gardé de côté pour évaluer les performances du réseau de neurones. En effet, ces données sont vierges et vous permettent d'évaluer la capacité du modèle à déterminer le chiffre d'un scan qu'il n'a jamais vu. 

Avec ce simple réseau de neurones vous devriez déjà atteindre une précision de 95% pour seulement 10 itérations ! Nous verrons par la suite que selon les architectures de réseau ou les processus d'apprentissage, nous pouvons encore améliorer les performances du modèle à déterminer un chiffre écrit à la main 

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)

## <font size="5" color='blue'> VI - Afficher l'apprentissage  </font>

Ici, vous utilisez les données recueillies lors de l'apprentissage du modèle pour voir comment évoluent les indices de performances. Sachez que les pertes du modèle ne sont pas le complémentaire de la précision donc les courbes ne seront pas totalement inversées mais seulement corrélées. 

On constate bien que comme l'apprentissage des données d'entrainement par le modèle se poursuit, la précision de l'indentification des chiffres augmente et les pertes associées aux calculs diminuent. Vers la 15e itération, on peut voir la courbe de validation se rapprocher et même s'inverser par rapport à celle d'apprentissage, c'est le phénomène d'over-fitting qui signifie que le modèle se concentre sur les données d'entrainement et peine à généraliser à de nouveaux exemples. 

La cellule du dessous permet d'enregistrer le modèle généré sous forme d'un fichier stockable sur votre ordinateur et ainsi de vous en resservir pour la suite.

In [ ]:
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 0.3])
plt.legend(loc='lower right')
plt.legend()
plt.grid()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Acuracy')
plt.ylim([0.9, 1])
plt.legend(loc='lower right')
plt.legend()
plt.grid()

plt.show()

In [ ]:
model.save('basicsmnistGPU.model')

## <font size="5" color='blue'> VII - Effectuer des prédictions  </font>

Cette partie est l'aboutissement du processus d'apprentissage : vous allez effectuer des prédictions sur des données inédites. Bon, comme toutes les données de la base de données MNIST ont été utilisées pour l'apprentissage, nous allons réutiliser un exemple déjà vu par l'algorithme pour faire des prédictions mais vous êtes libres d'écrire un chiffre à la main, de le scanner et mettre en forme puis de le faire lire par l'algorithme ^^.

Dans les dernières lignes remarquez que vous aurez accès à la prédiction du chiffre par le modèle par l'intermédiaire de la fonction argmax. En effet, rappelez-vous la couche de sortie du réseau de neurones est constituée de 10 neurones représentant les 10 chiffres normalisés en probabilité avec la fonction softmax. La prédiction du réseau de neurones correspond bien à cette configuration avec 10 valeurs entre 0 et 1 qui se somment pour donner 1. Pour récupérer la prédiction effective prenez donc l'index du neurone avec la plus grande activation : la fonction argmax.

In [ ]:
new_model = tf.keras.models.load_model('basicsmnistGPU.model')

(x_train, y_train), (x_test, y_test) = mnist.load_data()

predictions = new_model.predict(x_test/255.0)

from random import random 
idx = round(random() *10000)

from numpy import argmax
print('La prédiction pour le chiffre indéxé ' + str(idx) + ' est : ' + str(argmax(predictions[idx])))
print(argmax(predictions[idx])==y_test[idx])


plt.imshow(x_test[idx], cmap=plt.cm.binary)
print(y_test[idx])
print(predictions[idx])

Voilà ! Vous savez dorénavant programmer un algorithme de Machine Learning capable d'identifier un chiffre ! Cela peut vous paraître banal à première vue mais sachez qu'avant d'en arriver là, des générations d'informaticiens ont piétiné avant de mettre en place cet écosystème de Machine Learning. 

J'espère que vous aurez apprécié ce première application ML. Si c'est le cas, n'hésitez pas à partager et restez en contact pour plus de nouveautés. Ciao ! 


<a href="https://github.com/IAFutur/MachineLearning">IAFutur 04/2020</a>